# HW Урок 14. Transfer learning

In [ ]:
#!pip install pymorphy2[fast]
#!pip install pymorphy2
#!pip install annoy
#!pip install stop_words

In [ ]:
import string
import nltk
import annoy
import numpy as np
import tensorflow as tf
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tqdm import tqdm_notebook
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
!head -3 prepared_answers.txt

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [ ]:
bert = TFAutoModel.from_pretrained("setu4993/LaBSE")
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
q_len = []
a_len = []
with open("prepared_answers.txt", "r") as f:
    counter = 0
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        q_len.append(len(spls[0].split()))
        a_len.append(len(spls[1].split()))

<ipython-input-42-a19036b625f7>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


In [ ]:
np.median(q_len), np.mean(q_len), np.quantile(q_len, 0.75), np.quantile(q_len, 0.85), np.quantile(q_len, 0.95), np.max(q_len)

(12.0, 20.454707214215595, 21.0, 31.0, 63.0, 862)

In [ ]:
bert_index = annoy.AnnoyIndex(768 , 'angular')

index_map = {}
counter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue

        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])

        tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
        out_state = bert(**tok)[1].numpy()[0]
        bert_index.add_item(counter, out_state)
        counter += 1


bert_index.build(10)

In [ ]:
bert_index.save('bert_index')

In [ ]:
bert_index.build(10)

True

In [ ]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers = index.get_nns_by_vector(vector, 1, )
    return [index_map[i] for i in answers]

In [ ]:
get_response("какой навигатор выбрать?", bert_index, index_map)

['В семью или на работу?. \n']

In [ ]:
get_response("какие новые фильмы есть?", bert_index, index_map)

In [ ]:
get_response("какие новые фильмы посмотреть?", bert_index, index_map)

['Отпуск по обмену / The Holiday (2006 <br>Идеальное Рождество (Снежный глобус) / Snowglobe (2007) <br>Рождественский коттедж / Christmas Cottage (2008) <br>Ёлки (2010) <br>Новогодние сваты (2010) <br>Снежный принц / Snow Prince (2009) <br>Суженый-ряженый (2007) <br>Полночный поцелуй" (2008) <br>Ирония судьбы 2: Продолжение (2007) <br>История любви, или Новогодний розыгрыш (2008) <br>Карнавальная ночь-2, или 50 лет спустя (2006) <br>Новогодние сваты (2010) <br>Удивительная история 2008 <br>Идеальное Рождество 2007 <br>Реальная любовь 2003 <br>Эван Всемогущий 2007 <br>Четыре Рождества 2008 <br>Рождественский коттедж 2008 <br>Санта на продажу 2010 <br>Санта-Хрякус: Страшдественская сказка 2006 <br>Свадьба на Рождество 2006 <br>Нянька на Рождество 2010 <br>Добро пожаловать, или Соседям вход 2006 <br>Миллион на Рождество 2007 <br>Наконец-то Рождество 2008 <br>Наследство к Рождеству 2007 <br>Новогодняя засада (2008) <br>Новогодняя семейка (2008) <br>Новогодние приключения в июле (2008) <br>

In [ ]:
get_response("куда можно сходить?", bert_index, index_map)

['Это конкретное предложение? или просто так ...интересуешься?. \n',
 'Только не под себя.... \n']